# Citibike ML
In this example we use the [Citibike dataset](https://ride.citibikenyc.com/system-data). Citibike is a bicycle sharing system in New York City. Everyday users choose from 20,000 bicycles at 1300 stations around New York City.

To ensure customer satisfaction Citibike needs to predict how many bicycles will be needed at each station. Maintenance teams from Citibike will check each station and repair or replace bicycles. Additionally, the team will relocate bicycles between stations based on predicted demand. The business needs to be able to run reports of how many bicycles will be needed at a given station on a given day.

## Data Exploration and Modeling

Now we're to the fun part - the Data Science. Now that the data engineers have cleaned and loaded the data to the `trips` table, we can begin our model development. For this, we will leverage Snowpark to do the **feature preparation and exploratory analysis**.   This dataset is initially ~100 million rows and is likely too large to fit into memory on our local machine or even a reasonable sized single VM in the cloud. The Snowpark Python client-side Dataframe API 
allows us to push-down most of the computation for preparation and feature engineering to Snowpark. For security and goverance reasons we can read data into memory for model training and inference but no intermediate data products can be stored outside of Snowflake.  

For this demo flow we will assume that the organization has the following **policies and processes** :   
-**Dev Tools**: The data scientist can develop in their tool of choice (ie. DataRobot, Dataiku, H2o, Sagemaker, AzureML, etc.).  Snowpark Python makes it possible to use any environment with a python kernel.  For the sake of a demo we will use Jupyter.  
-**Data Governance**: To preserve customer privacy no data can be stored locally.  The ingest system may store data temporarily but it must be assumed that, in production, the ingest system will not preserve intermediate data products between runs. Snowpark Python allows the user to push-down all operations to Snowflake and bring the code to the data.   
-**Automation**: Although the data scientist may use any IDE or notebooks for development purposes the final product must be python code at the end of the work stream.  Well-documented, modularized code is necessary for good ML operations and to interface with the company's CI/CD and orchestration tools.  
-**Compliance**: Any ML models must be traceable back to the original data set used for training.  The business needs to be able to easily remove specific user data from training datasets and retrain models. 

Input: Data in `trips` table.   
Output: Feature engineering logic.  Train function.  Predict function.

For this demo we will rewind in time and assume that it is March 1, 2020.  With the bulk ingest we have 94M records from 2013 to March 2020.

In [ ]:
#today = '2020-03-01'

In [ ]:
#!pip install -q matplotlib seaborn pytorch-tabnet

### 1. Load the Credentials


In [ ]:
from dags.snowpark_connection import snowpark_connect
session, state_dict = snowpark_connect()

### 2. Exploratory Data Analysis

In [ ]:
import snowflake.snowpark as snp
from snowflake.snowpark import functions as F
from snowflake.snowpark import types as T

import pandas as pd
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import mean_squared_error

# import logging
# logging.basicConfig(level=logging.WARN)
# logging.getLogger().setLevel(logging.DEBUG)

In [ ]:
trips_table_name = state_dict['trips_table_name']
holiday_table_name = 'HOLIDAYS'
precip_table_name = 'WEATHER'

In [ ]:
session.table(trips_table_name).select(F.min('STARTTIME'), F.max('STARTTIME')).show()
session.table(trips_table_name).count()

This is too large to fit in memory on my local system.  Lets summarize trips to daily resolution and inspect the first ten rows

In [ ]:
snowdf = session.table(trips_table_name)
snowdf.with_column('DATE', F.to_date('STARTTIME')).group_by('DATE').count().sort('DATE').show(10)

Once we aggregate the data at the day level we have a small enough dataset to fit in memory.  But we may want to provide a more granular time series (ie. hour or minute-level) or perhaps our data will grow considerably over time.  In either case we can't rely on in-memory computation and will want to push-down as much computation as possible to Snowflake.  
  
For exploration purposes we can see a good daily and annual seasonality in the historical data.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df = snowdf.with_column("date", F.to_date("STARTTIME")).group_by("date").count().sort("date").to_pandas()
df.head()

plt.figure(figsize=(15, 8))
ax = sns.lineplot(x='DATE', y='COUNT', data=df)

We may not be able to get a good model that can predict across ALL stations.  Lets start with just the busiest stations(s).

In [ ]:
snowdf.filter(F.col('START_STATION_ID').is_not_null()) \
      .group_by('START_STATION_ID') \
      .count() \
      .sort('COUNT', ascending=False).show()

Initially we will build with the busiest station "Central Park S & 6 Ave" which is STATION_ID=519

In [ ]:
top_stations = snowdf.filter(F.col('START_STATION_ID').is_not_null()) \
                                                   .groupBy('START_STATION_ID') \
                                                   .count() \
                                                   .sort('COUNT', ascending=False) \
                                                   .toPandas()['START_STATION_ID'].values.tolist()
top_stations[0:10]

In [ ]:
df = snowdf.filter(F.col('START_STATION_ID') == top_stations[0]) \
      .withColumn('DATE', 
                  F.call_builtin('DATE_TRUNC', ('DAY', F.col('STARTTIME')))) \
      .groupBy('DATE') \
      .count() \
      .sort('DATE').toPandas()

df.head()

In [ ]:
plt.figure(figsize=(15, 8))
ax = sns.lineplot(x='DATE', y='COUNT', data=df)

To start building the features for a regression model we might start with daily, weekly, monthly, quarterly lag features. 
  
Snowpark has many [functions](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/_autosummary/snowflake.snowpark.functions.html) for things like transformation, statistical analysis, etc. We will use the `lag()` window function to generate lag features.  
  
In addition to the rich set of Snowpark functions, users can also tap into the wealth of [Snowflake built-in functions](https://docs.snowflake.com/en/sql-reference/functions-all.html) using the `call_builtin()` function. 

Here we will generate our features in a function which we may need to add to over time.  In the end the function provides a running list of transformations to build our feature pipeline for inference and training.

In [ ]:
def generate_features(snowdf):
    
    snowdf = snowdf.select(F.to_date(F.col('STARTTIME')).alias('DATE'),
                           F.col('START_STATION_ID').alias('STATION_ID'))\
                   .group_by(F.col('STATION_ID'), F.col('DATE'))\
                   .count()
    
    #Impute missing values for lag columns using mean of the previous period.
    mean_1 = round(snowdf.sort('DATE').limit(1).select(F.mean('COUNT')).collect()[0][0])
    mean_7 = round(snowdf.sort('DATE').limit(7).select(F.mean('COUNT')).collect()[0][0])
    mean_30 = round(snowdf.sort('DATE').limit(30).select(F.mean('COUNT')).collect()[0][0])
    mean_365 = round(snowdf.sort('DATE').limit(365).select(F.mean('COUNT')).collect()[0][0])

    date_win = snp.Window.order_by('DATE')

    snowdf = snowdf.with_column('LAG_1', F.lag('COUNT', offset=1, default_value=mean_1) \
                                         .over(date_win)) \
                   .with_column('LAG_7', F.lag('COUNT', offset=7, default_value=mean_7) \
                                         .over(date_win)) \
                   .with_column('LAG_30', F.lag('COUNT', offset=30, default_value=mean_30) \
                                         .over(date_win)) \
                   .with_column('LAG_365', F.lag('COUNT', offset=365, default_value=mean_365) \
                                         .over(date_win)) \
                   .na.drop()
    return snowdf

In [ ]:
train_snowdf = generate_features(snowdf.filter(F.col('START_STATION_ID') == top_stations[0]))
train_snowdf.show()

### 3. Model Development and Experimentation
We will use [pytorch_tabnet](https://github.com/dreamquark-ai/tabnet) from the 2019 [tabnet paper](https://arxiv.org/pdf/1908.07442.pdf) by Arik, S. O. and Pfister, T.  Tabnet is a powerful deep learning framework for attentive, interpretable learning on tabular data.  Rather than substantial focus on hyper-parameter optimization  we will start with an initial set of hyper-parameters and focus on iterating over input features for now.  
  
Rather than a random split of training/validation data we will split the training dataset using a `cutpoint`.  For example we will save the final 365 days of the dataset as validation and train with the remaining data.

In [ ]:
def train(X, y, cutpoint=365, cat_idxs=[], cat_dims=[]):    
    X_valid = X[-cutpoint:]
    y_valid = y[-cutpoint:]
    X_train = X[:-cutpoint]
    y_train = y[:-cutpoint]

    from pytorch_tabnet.tab_model import TabNetRegressor
    import pandas as pd

    max_epochs = 100
    
    batch_df = pd.DataFrame(range(2,65,2), columns=['batch_size'])
    batch_df['batch_remainder'] = len(X_train)%batch_df['batch_size']
    optimal_batch_size=int(batch_df['batch_size'].where(batch_df['batch_remainder']==batch_df['batch_remainder'].min()).max())
    
    print('Selected batch size '+str(optimal_batch_size)+' for input data size: '+str(len(X_train)))
    
    regression_model = TabNetRegressor(cat_idxs=cat_idxs, cat_dims=cat_dims)

    regression_model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        max_epochs=max_epochs,
        patience=10,
        batch_size=optimal_batch_size, 
        virtual_batch_size=optimal_batch_size/2,
        num_workers=0,
        drop_last=True)
    
    return regression_model

def predict(model, X):
    y_hat = model.predict(X).reshape(-1)
    return y_hat
    
def plot(df, x_lab:str, y_true_lab:str, y_pred_lab:str):
    plt.figure(figsize=(15, 8))
    df = pd.melt(df, id_vars=[x_lab], value_vars=[y_true_lab, y_pred_lab])
    ax = sns.lineplot(x=x_lab, y='value', hue='variable', data=df)

We will create a pandas dataframe using toPandas() which will generate the features as a pyarrow dataset and efficiently read them into memory in pandas locally.  
  
Let's train our first model to get a baseline. 

In [ ]:
def train_predict(df, cat_idxs=[], cat_dims=[]):
    target = ['COUNT']
    feature_columns = [feature.replace('\"', '') for feature in df.columns]
    feature_columns.remove(target[0])
    feature_columns.remove('DATE')
    feature_columns.remove('STATION_ID')

    model = train(df[feature_columns].astype(float).values, 
                  df[target].values,
                  cat_idxs=cat_idxs, 
                  cat_dims=cat_dims)
    df['Y_PRED'] = predict(model, df[feature_columns].astype(float).values).astype('int')

    explain_df = pd.DataFrame(model.explain(df[feature_columns].astype(float).values)[0], 
                          columns = feature_columns).add_prefix('EXPL_').round(2)
    df = pd.concat([df.set_index('DATE').reset_index(), explain_df], axis=1)
    
    MSE = mean_squared_error(y_pred=df['Y_PRED'], y_true=df[target])
    display("Error for training dataset is: "+str(MSE))
    
    return df, model, feature_columns

In [ ]:
df = train_snowdf.sort('DATE', ascending=True).to_pandas()
df, model, feature_columns = train_predict(df)

In [ ]:
plot(df, 'DATE', 'COUNT', 'Y_PRED')

#### What if we add US holidays as a feature?
Pandas has great support for holiday lists.  We can generate a pandas dataframe and then upload it as a table to create a new feature.

In [ ]:
def generate_holiday_df(session, start_date, end_date):
    from snowflake.snowpark import functions as F 
    import pandas as pd
    from pandas.tseries.holiday import USFederalHolidayCalendar

    cal = USFederalHolidayCalendar()
    holiday_list = pd.DataFrame(cal.holidays(start=start_date, end=end_date), columns=['DATE'], dtype=str)
    holiday_df = session.create_dataframe(holiday_list) \
                        .toDF(holiday_list.columns[0]) \
                        .with_column("HOLIDAY", F.lit(1))
    return holiday_df
    
start_date, end_date = train_snowdf.select(F.min('DATE'), F.max('DATE')).collect()[0][0:2]

holiday_df = generate_holiday_df(session, start_date=start_date, end_date=end_date)
holiday_df.show(3)

Here we create an in-memory instance of the holiday dataframe.  In production we probably want to materialize this feature as a table or view.  We'll see later how that is easy to do but for now we have a function to generate it and join it to our training dataframe as a one-hot feature.

Every time we join there is a possibility of losing data from our feature set if the join column doesn't cover the same dates.  So its good to check.

In [ ]:
train_snowdf = train_snowdf.join(holiday_df, 'DATE', join_type='left') \
                           .na.fill({'HOLIDAY':0}) \
                           .sort('DATE', ascending=True)
train_snowdf.show()

Now when we train we need to specify this new holiday feature as a categorical feature.  Its already encoded (by definition) so nothing to add there.

In [ ]:
df = train_snowdf.sort('DATE', ascending=True).to_pandas()
df, model, feature_columns = train_predict(df, cat_idxs=[-1], cat_dims=[2])

In [ ]:
plot(df, 'DATE', 'COUNT', 'Y_PRED')

#### Enhancing Model Accuracy With Weather Data

Let's see how we can make our model even better with weather data. Its likely that weather and the amount of precipitation on the day will be an important signal for our model. 
  
We might start by downloading weather data or setting up an API.  But this is a lot of work when we want to build inference pipelines later.

In [ ]:
def generate_precip_df(session, start_date, end_date):
    from snowflake.snowpark import functions as F 
    import pandas as pd

    tempdf = pd.read_csv('./include/weather.csv')
    tempdf['DATE']=pd.to_datetime(tempdf['dt_iso'].str.replace(' UTC', ''), 
                                 format='%Y-%m-%d %H:%M:%S %z', 
                                 utc=True).dt.tz_convert('America/New_York').dt.date
    tempdf = tempdf.groupby('DATE', as_index=False).agg(PRECIP=('rain_1h', 'mean')).round()

    precip_df = session.create_dataframe(tempdf).filter((F.col('DATE') >= start_date) &
                                                        (F.col('DATE') <= end_date))\
                       .fillna({'PRECIP':0})
    return precip_df

####  Optionally use the [Snowflake Data Marketplace](https://www.snowflake.com/data-marketplace/?_sft_dataset-category=weather) & Data Sharing
The Snowflake marketplace would allow us to very easily access weather data and join it with the trips data.  Not only is this faster and more scalable than building from a csv file or API, it is extremely easy to setup an operational pipeline to keep it fresh.  
  
For this demo we will continue to use the csv format as we cannot be sure that each user has access to the marketplace data already setup.

In [ ]:
# def generate_precip_df(start_date, end_date):
#     precip_df = session.table('CITIBIKE_WEATHER_V3.WEATHER_V3.HISTORY_DAY') \
#                          .filter(F.col('POSTAL_CODE') == '08562')\
#                          .select(F.col('DATE_VALID_STD').alias('DATE'), 
#                                  F.col('TOT_PRECIPITATION_IN').alias('PRECIP'))
#     #precip_df = session.table('weather')

#     return precip_df

We add a precip feature by joining the precipitation dataframe.

In [ ]:
start_date, end_date = train_snowdf.select(F.min('DATE'), F.max('DATE')).collect()[0][0:2]

precip_df = generate_precip_df(session, start_date=start_date, end_date=end_date)

train_snowdf = train_snowdf.join(precip_df, 'DATE', 'inner').sort('DATE', ascending=True)

train_snowdf.show()

Again, check the date range after the join.

In [ ]:
train_snowdf.select(F.min('DATE'), F.max('DATE')).show()

In [ ]:
df = train_snowdf.sort('DATE', ascending=True).to_pandas()
df, model, feature_columns = train_predict(df, cat_idxs=[-2], cat_dims=[2])

In [ ]:
plot(df, 'DATE', 'COUNT', 'Y_PRED')

#### Lets look at the feature importance
One reason we chose tabnet for this analysis is the built-in abilities for explainability.

In [ ]:
pd.DataFrame([model.feature_importances_], columns=feature_columns)

#### Lets consolidate and upate our feature functions and save them for the ML engineering team to operationalize.
Based on the feature importance above we will go with 1 and 7 day lags only.

In [ ]:
%%writefile dags/feature_engineering.py

def generate_holiday_df(session, start_date, end_date):
    from snowflake.snowpark import functions as F 
    import pandas as pd
    from pandas.tseries.holiday import USFederalHolidayCalendar

    cal = USFederalHolidayCalendar()
    holiday_list = pd.DataFrame(cal.holidays(start=start_date, end=end_date), columns=['DATE'], dtype=str)
    holiday_df = session.create_dataframe(holiday_list) \
                        .toDF(holiday_list.columns[0]) \
                        .with_column("HOLIDAY", F.lit(1))
    return holiday_df

def generate_precip_df(session, start_date, end_date):
    from snowflake.snowpark import functions as F 
    import pandas as pd

    tempdf = pd.read_csv('./include/weather.csv')
    tempdf['DATE']=pd.to_datetime(tempdf['dt_iso'].str.replace(' UTC', ''), 
                                 format='%Y-%m-%d %H:%M:%S %z', 
                                 utc=True).dt.tz_convert('America/New_York').dt.date
    tempdf = tempdf.groupby('DATE', as_index=False).agg(PRECIP=('rain_1h', 'mean')).round()

    precip_df = session.create_dataframe(tempdf).filter((F.col('DATE') >= start_date) &
                                                        (F.col('DATE') <= end_date))\
                       .fillna({'PRECIP':0})
    return precip_df

def generate_features(session, input_df, holiday_table_name, precip_table_name):
    import snowflake.snowpark as snp
    from snowflake.snowpark import functions as F 
    
    start_date, end_date = input_df.select(F.min('STARTTIME'), F.max('STARTTIME')).collect()[0][0:2]

    holiday_df = session.table(holiday_table_name)
    try: 
        _ = holiday_df.columns()
    except:
        holiday_df = generate_holiday_df(session=session,
                                         start_date=start_date,
                                         end_date=end_date)
        
    precip_df = session.table(precip_table_name)
    try: 
        _ = precip_df.columns()
    except:
        precip_df = generate_precip_df(session=session,
                                       start_date=start_date,
                                       end_date=end_date)

    feature_df = input_df.select(F.to_date(F.col('STARTTIME')).alias('DATE'),
                                 F.col('START_STATION_ID').alias('STATION_ID'))\
                         .replace({'NULL': None}, subset=['STATION_ID'])\
                         .groupBy(F.col('STATION_ID'), F.col('DATE'))\
                         .count()

    #Impute missing values for lag columns using mean of the previous period.
    mean_1 = round(feature_df.sort('DATE').limit(1).select(F.mean('COUNT')).collect()[0][0])
    mean_7 = round(feature_df.sort('DATE').limit(7).select(F.mean('COUNT')).collect()[0][0])
    mean_365 = round(feature_df.sort('DATE').limit(365).select(F.mean('COUNT')).collect()[0][0])

    date_win = snp.Window.orderBy('DATE')

    feature_df = feature_df.with_column('LAG_1', F.lag('COUNT', offset=1, default_value=mean_1) \
                                         .over(date_win)) \
                           .with_column('LAG_7', F.lag('COUNT', offset=7, default_value=mean_7) \
                                         .over(date_win)) \
                           .with_column('LAG_365', F.lag('COUNT', offset=365, default_value=mean_365) \
                                         .over(date_win)) \
                           .join(holiday_df, 'DATE', join_type='left').na.fill({'HOLIDAY':0}) \
                           .join(precip_df, 'DATE', 'inner') \
                          .na.drop() 

    return feature_df

Make one final test of feature engineering with our new code then train the final model.

In [ ]:
from dags.feature_engineering import generate_features

snowdf = session.table(trips_table_name).filter(F.col('START_STATION_ID') == top_stations[0])

train_snowdf = generate_features(session=session, 
                                 input_df=snowdf, 
                                 holiday_table_name=holiday_table_name, 
                                 precip_table_name=precip_table_name)

df = train_snowdf.sort('DATE', ascending=True).toPandas()
df, model, feature_columns = train_predict(df, cat_idxs=[-2], cat_dims=[2])

In [ ]:
plot(df, 'DATE', 'COUNT', 'Y_PRED')

In [ ]:
pd.DataFrame([model.feature_importances_], columns=feature_columns)

### Inference Testing and Evaluation
We will want to be able to look back over an period of time to check model performance over a specific period.

In [ ]:
def generate_past_df(session, trips_table_name, station_id, pred_period, holiday_table_name, precip_table_name):
    from dags.feature_engineering import generate_features
    
    end_date = session.table(trips_table_name).select(F.max('STARTTIME')).collect()[0][0]
    pastdf = session.table(trips_table_name)\
                    .filter(F.col('START_STATION_ID') == station_id)\
                    .filter(F.to_date('STARTTIME') >= F.dateadd('DAY', 
                                                                F.lit(-7+pred_period), 
                                                                F.lit(end_date)))
    newdf = generate_features(session=session, 
                              input_df=pastdf, 
                              holiday_table_name=holiday_table_name,
                              precip_table_name=precip_table_name)

    newdf = newdf.filter(F.to_date('DATE') >= F.dateadd(pred_interval,
                                                        F.lit(pred_period),
                                                        F.lit(end_date)))
  
    return newdf

Lets see how our model performed for the previous 90 days

In [ ]:
pred_period=-90
pred_interval='DAY'
past_df = generate_past_df(session=session, 
                           trips_table_name=trips_table_name, 
                           station_id=top_stations[0], 
                           pred_period=pred_period, 
                           holiday_table_name=holiday_table_name,
                           precip_table_name=precip_table_name)

target = ['COUNT']

df = past_df.sort('DATE', ascending=True).toPandas()

df['Y_PRED'] = predict(model, df[feature_columns].astype(float).values).astype('int')

# MSE = mean_squared_error(y_pred=df['Y_PRED'], y_true=df[target])
# display("Error for "+str(prediction_period)+" days from "+today+" is: "+str(MSE))

In [ ]:
past_df.select(F.min('DATE'), F.max('DATE')).show()

In [ ]:
plot(df, 'DATE', 'COUNT', 'Y_PRED')